# Parallel Worker Dispatcher

## Detailed Analysis

This script serves as a unified entry point for the parallel data generation process. It is designed to be called by the `joblib` library in the main orchestration script. The function acts as a dispatcher that routes compute tasks to the appropriate physics generation engine—either Hadronic or Quark—based on the instructions provided in the task tuple.

This abstraction layer allows the main pipeline to submit a mixed list of tasks without needing to manage separate parallel pools for different physics models.

## Code Walkthrough

### Task Unpacking and Dispatch
The `run_worker_wrapper` function accepts a single task tuple and a shared resource (`baselines`). The task metadata is unpacked to determine the execution mode.

```python
def run_worker_wrapper(task_tuple, baselines):
    # Unpack task metadata
    mode, batch_size, seed, batch_idx = task_tuple
    
    if mode == 'hadronic':
        return worker_hadronic_gen(batch_size, baselines, seed, batch_idx)
    else:
        return worker_quark_gen(batch_size, seed, batch_idx)
```

**Hadronic Dispatch:**
If the mode is `'hadronic'`, the `worker_hadronic_gen` function is called. The `baselines` dictionary is passed to this worker because the hadronic generation method ("Smart Inverse Sampling") requires the maximum mass of parent models to calculate homologous scaling factors ($\alpha$).

**Quark Dispatch:**
If the mode is not hadronic, the `worker_quark_gen` function is called. The `baselines` argument is not used here because the Quark Star model (Generalized CFL) generates Equations of State parametrically from microphysical constants ($B, \Delta, m_s$) rather than scaling existing nuclear models.